In [2]:

import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  # accessor needed to parse crs
import calendar
import argparse
import seaborn as sns
from shapely.geometry import Point
import geopandas
from geodatasets import get_path
import glob
import os

In [3]:
dir_path = '/group/moniergrp/TEM_Large_Ensemble/output_files/' 
region = 'region_1/'

In [3]:
tem_output_files = [
'AVAILN_future.csv'
,'GPP_future.csv'
,'LAI_future.csv'
,'NCE_future.csv'
,'NEP_future.csv'
,'NETNMIN_future.csv'
,'NPP_future.csv'
,'SOILORGC_future.csv'
,'VEGC_future.csv'
,'VEGINNPP_future.csv'
,'VSM_future.csv'
]

In [5]:
i=0
tt = pd.read_csv(dir_path+region+tem_output_files[i])
tt.columns

Index(['model', 'scenario', 'grid_group', 'lon', 'lat', 'variable',
       'forest_type', 'current_veg', 'year', 'silt_clay',
       'stand_age_interval_min', 'cohort_area', 'land_area', 'monthly_mean',
       'cohort_annual_change_delta', 'cohort_annual_change_percent'],
      dtype='object')

,model,scenario,grid_group,lon,lat,variable,forest_type,current_veg,year,silt_clay,stand_age_interval_min,cohort_area,land_area,monthly_mean,cohort_annual_change_delta,cohort_annual_change_percent
0,AWIxxCMxx1xx1xxMR,ssp126,520,-122.5,47.5,AVAILN,Temperate Coniferous Forests,9,2015,53.75,0,41,29092,6547.20317,0.00000,0.00000
1,AWIxxCMxx1xx1xxMR,ssp126,520,-122.5,47.5,AVAILN,Temperate Coniferous Forests,9,2015,53.75,5,53,24936,3227.46868,0.00000,0.00000
2,AWIxxCMxx1xx1xxMR,ssp126,520,-122.5,47.5,AVAILN,Temperate Coniferous Forests,9,2015,53.75,10,56,29092,2853.77786,0.00000,0.00000
3,AWIxxCMxx1xx1xxMR,ssp126,520,-122.5,47.5,AVAILN,Temperate Coniferous Forests,9,2015,53.75,15,81,37404,2875.29568,0.00000,0.00000
4,AWIxxCMxx1xx1xxMR,ssp126,520,-122.5,47.5,AVAILN,Temperate Coniferous Forests,9,2015,53.75,20,84,31170,1861.78869,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21325015,CanESM5,ssp585,585,-120.0,54.0,AVAILN,Boreal Forest,4,2100,56.17,90,33,10836,367.43030,-8.83818,-0.02439
21325016,CanESM5,ssp585,585,-120.0,54.0,AVAILN,Boreal Forest,4,2100,56.17,95,38,9030,390.66026,-7.82684,-0.02017
21325017,CanESM5,ssp585,585,-120.0,54.0,AVAILN,Boreal Forest,4,2100,56.17,100,161,46956,433.12174,-6.40534,-0.01509
21325018,CanESM5,ssp585,585,-120.0,54.0,AVAILN,Boreal Forest,4,2100,56.17,125,98,45150,479.72398,-4.98337,-0.01078


In [21]:
for i in range(len(tem_output_files)):
    tem_var = pd.read_csv(dir_path+region+tem_output_files[i])
    var = tem_var.loc[0,'variable'].replace(" ", "")
    tem_var=tem_var.rename(columns={'monthly_mean':var+'_monthly_mean'
                            ,'cohort_annual_change_delta':var+'_cohort_annual_change_delta'
                            ,'cohort_annual_change_percent':var+'_cohort_annual_change_percent'})

    tem_var.drop(columns='variable', inplace=True)    
    if (i==0):
        tem_var_merge = tem_var
    else:
        tem_var_merge = tem_var_merge.merge(tem_var, on = ['model', 'scenario', 'grid_group', 'lon', 'lat', 
       'forest_type', 'current_veg', 'year', 'silt_clay',
       'stand_age_interval_min', 'cohort_area', 'land_area'])
    print(len(tem_var_merge))



21325020
21325020
21325020
21325020
21325020
21325020
21325020
21325020
21325020
21325020
21325020


In [4]:
tem_output_files = [
'AVAILN_pre_data.csv'
,'GPP_pre_data.csv'
,'LAI_pre_data.csv'
,'NCE_pre_data.csv'
,'NEP_pre_data.csv'
,'NETNMIN_pre_data.csv'
,'NPP_pre_data.csv'
,'SOILORGC_pre_data.csv'
,'VEGC_pre_data.csv'
,'VEGINNPP_pre_data.csv'
,'VSM_pre_data.csv'
]

for i in range(len(tem_output_files)):
    tem_var = pd.read_csv(dir_path+region+tem_output_files[i])
    var = tem_var.loc[0,'variable'].replace(" ", "")
    tem_var=tem_var.rename(columns={'monthly_mean':var+'_monthly_mean'
                            ,'cohort_annual_change_delta':var+'_cohort_annual_change_delta'
                            ,'cohort_annual_change_percent':var+'_cohort_annual_change_percent'})

    tem_var.drop(columns='variable', inplace=True)    
    if (i==0):
        tem_var_merge = tem_var
    else:
        tem_var_merge = tem_var_merge.merge(tem_var, on = ['model', 'grid_group', 'lon', 'lat', 
       'forest_type', 'current_veg', 'year', 'silt_clay',
       'stand_age_interval_min', 'cohort_area', 'land_area'])
    print(len(tem_var_merge))



1743056
1743056
1743056
1743056
1743056
1743056
1743056
1743056
1743056
1743056
1743056


In [8]:
tem_var_merge['AVAILN_monthly_mean'] = tem_var_merge['AVAILN_monthly_mean'].round(2)
tem_var_merge['AVAILN_cohort_annual_change_delta']  = tem_var_merge['AVAILN_cohort_annual_change_delta'].round(2)
tem_var_merge['AVAILN_cohort_annual_change_percent']  = tem_var_merge['AVAILN_cohort_annual_change_percent'].round(2)
tem_var_merge['GPP_monthly_mean']  = tem_var_merge['GPP_monthly_mean'].round(2)
tem_var_merge['GPP_cohort_annual_change_delta']  = tem_var_merge['GPP_cohort_annual_change_delta'].round(2)
tem_var_merge['GPP_cohort_annual_change_percent']  = tem_var_merge['GPP_cohort_annual_change_percent'].round(2)
tem_var_merge['LAI_monthly_mean']  = tem_var_merge['LAI_monthly_mean'].round(2)
tem_var_merge['LAI_cohort_annual_change_delta']  = tem_var_merge['LAI_cohort_annual_change_delta'].round(2)
tem_var_merge['LAI_cohort_annual_change_percent']  = tem_var_merge['LAI_cohort_annual_change_percent'].round(2)
tem_var_merge['NCE_monthly_mean']  = tem_var_merge['NCE_monthly_mean'].round(2)
tem_var_merge['NCE_cohort_annual_change_delta']  = tem_var_merge['NCE_cohort_annual_change_delta'].round(2)
tem_var_merge['NCE_cohort_annual_change_percent']  = tem_var_merge['NCE_cohort_annual_change_percent'].round(2)
tem_var_merge['NEP_monthly_mean']  = tem_var_merge['NEP_monthly_mean'].round(2)
tem_var_merge['NEP_cohort_annual_change_delta']  = tem_var_merge['NEP_cohort_annual_change_delta'].round(2)
tem_var_merge['NEP_cohort_annual_change_percent']  = tem_var_merge['NEP_cohort_annual_change_percent'].round(2)
tem_var_merge['NETNMIN_monthly_mean']  = tem_var_merge['NETNMIN_monthly_mean'].round(2)
tem_var_merge['NETNMIN_cohort_annual_change_delta']  = tem_var_merge['NETNMIN_cohort_annual_change_delta'].round(2)
tem_var_merge['NETNMIN_cohort_annual_change_percent']  = tem_var_merge['NETNMIN_cohort_annual_change_percent'].round(2)
tem_var_merge['NPP_monthly_mean']  = tem_var_merge['NPP_monthly_mean'].round(2)
tem_var_merge['NPP_cohort_annual_change_delta']  = tem_var_merge['NPP_cohort_annual_change_delta'].round(2)
tem_var_merge['NPP_cohort_annual_change_percent']  = tem_var_merge['NPP_cohort_annual_change_percent'].round(2)
tem_var_merge['SOILORGC_monthly_mean']  = tem_var_merge['SOILORGC_monthly_mean'].round(2)
tem_var_merge['SOILORGC_cohort_annual_change_delta']  = tem_var_merge['SOILORGC_cohort_annual_change_delta'].round(2)
tem_var_merge['SOILORGC_cohort_annual_change_percent']  = tem_var_merge['SOILORGC_cohort_annual_change_percent'].round(2)
tem_var_merge['VEGC_monthly_mean']  = tem_var_merge['VEGC_monthly_mean'].round(2)
tem_var_merge['VEGC_cohort_annual_change_delta']  = tem_var_merge['VEGC_cohort_annual_change_delta'].round(2)
tem_var_merge['VEGC_cohort_annual_change_percent']  = tem_var_merge['VEGC_cohort_annual_change_percent'].round(2)
tem_var_merge['VEGINNPP_monthly_mean']  = tem_var_merge['VEGINNPP_monthly_mean'].round(2)
tem_var_merge['VEGINNPP_cohort_annual_change_delta']  = tem_var_merge['VEGINNPP_cohort_annual_change_delta'].round(2)
tem_var_merge['VEGINNPP_cohort_annual_change_percent']  = tem_var_merge['VEGINNPP_cohort_annual_change_percent'].round(2)
tem_var_merge['VSM_monthly_mean']  = tem_var_merge['VSM_monthly_mean'].round(2)
tem_var_merge['VSM_cohort_annual_change_delta']  = tem_var_merge['VSM_cohort_annual_change_delta'].round(2)
tem_var_merge['VSM_cohort_annual_change_percent']  = tem_var_merge['VSM_cohort_annual_change_percent'].round(2)
        


In [6]:
tem_var_merge.to_csv(
    dir_path+region+'pre_data_tem_outputs_merge.csv'
,index=False)
del tem_var_merge

In [7]:

tem_output_files = [
'AVAILN_historical.csv'
,'GPP_historical.csv'
,'LAI_historical.csv'
,'NCE_historical.csv'
,'NEP_historical.csv'
,'NETNMIN_historical.csv'
,'NPP_historical.csv'
,'SOILORGC_historical.csv'
,'VEGC_historical.csv'
,'VEGINNPP_historical.csv'
,'VSM_historical.csv'
]


for i in range(len(tem_output_files)):
    tem_var = pd.read_csv(dir_path+region+tem_output_files[i])
    var = tem_var.loc[0,'variable'].replace(" ", "")
    tem_var=tem_var.rename(columns={'monthly_mean':var+'_monthly_mean'
                            ,'cohort_annual_change_delta':var+'_cohort_annual_change_delta'
                            ,'cohort_annual_change_percent':var+'_cohort_annual_change_percent'})

    tem_var.drop(columns='variable', inplace=True)    
    if (i==0):
        tem_var_merge = tem_var
    else:
        tem_var_merge = tem_var_merge.merge(tem_var, on = ['model', 'grid_group', 'lon', 'lat', 
       'forest_type', 'current_veg', 'year', 'silt_clay',
       'stand_age_interval_min', 'cohort_area', 'land_area'])
    print(len(tem_var_merge))



4736830
4736830
4736830
4736830
4736830
4736830
4736830
4736830
4736830
4736830
4736830


In [9]:
tem_var_merge.to_csv(
    dir_path+region+'historical_tem_outputs_merge.csv'
,index=False)
del tem_var_merge